# Lab 5

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [2]:

import random

class Card:
    def __init__(self, suit, rank):
        self.suit, self.rank = suit, rank
    def __str__(self): return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_sets):
        self.num_sets, self.cards = num_sets, []
        [self.cards.extend(Card(suit, rank) for suit in ['Hearts','Diamonds','Clubs','Spades'] for rank in ['2','3','4','5','6','7','8','9','10','Jack','Queen','King','Ace']) for _ in range(num_sets)]
        self.plastic_card = random.choice(self.cards)
        self.cards.remove(self.plastic_card)

    def shuffle(self):
        random.shuffle(self.cards)

    def draw(self):
        if not self.cards: raise ValueError("The deck is empty. Cannot draw a card.")
        return self.cards.pop()

    def deal_plastic_card(self): return self.plastic_card

    def replace_plastic_card(self):
        self.cards.append(self.plastic_card)
        self.plastic_card = random.choice(self.cards)
        self.cards.remove(self.plastic_card)

# Example Usage
num_sets = 2
deck = Deck(num_sets)

deck.shuffle()
drawn_card = deck.draw()
print(f"Drawn Card: {drawn_card}")

plastic_card = deck.deal_plastic_card()
print(f"Plastic Card: {plastic_card}")

deck.replace_plastic_card()
deck.shuffle()

drawn_card = deck.draw()
print(f"Drawn Card: {drawn_card}")



Drawn Card: 6 of Clubs
Plastic Card: Jack of Diamonds
Drawn Card: Queen of Clubs


2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

In [7]:
import random

# Define the cards and deck
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_decks):
        self.cards = [Card(suit, rank) for _ in range(num_decks) for suit in suits for rank in ranks]
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def deal_card(self):
        if len(self.cards) == 0:
            raise ValueError("No cards left in the deck.")
        return self.cards.pop()

class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def get_value(self):
        value = 0
        ace_count = 0

        for card in self.cards:
            if card.rank.isdigit():
                value += int(card.rank)
            elif card.rank in ['Jack', 'Queen', 'King']:
                value += 10
            else:
                ace_count += 1
                value += 11

        while value > 21 and ace_count:
            value -= 10
            ace_count -= 1

        return value

# Player classes
class Player:
    def __init__(self):
        self.hand = Hand()

    def receive_card(self, card):
        self.hand.add_card(card)

class HumanPlayer(Player):
    def make_move(self):
        return input("Do you want to hit or stand? (Enter 'hit' or 'stand'): ").lower()

class Dealer(Player):
    def play_turn(self, deck):
        while self.hand.get_value() < 17:
            self.receive_card(deck.deal_card())

# Game Logic
def blackjack():
    num_decks = 6
    deck = Deck(num_decks)

    player = HumanPlayer()
    dealer = Dealer()

    # Initial deal
    for _ in range(2):
        player.receive_card(deck.deal_card())
        dealer.receive_card(deck.deal_card())

    # Player's turn
    while True:
        move = player.make_move()
        if move == 'hit':
            player.receive_card(deck.deal_card())
            if player.hand.get_value() > 21:
                print("You busted! Dealer wins.")
                return
        else:
            break

    # Dealer's turn
    dealer.play_turn(deck)

    # Determine winner
    player_value = player.hand.get_value()
    dealer_value = dealer.hand.get_value()

    print(f"Your hand: {[str(card) for card in player.hand.cards]} ({player_value})")
    print(f"Dealer's hand: {[str(card) for card in dealer.hand.cards]} ({dealer_value})")

    if dealer_value > 21 or player_value > dealer_value:
        print("You win!")
    elif dealer_value > player_value:
        print("Dealer wins!")
    else:
        print("It's a tie!")

# Start the game
blackjack()


Do you want to hit or stand? (Enter 'hit' or 'stand'): hit
Do you want to hit or stand? (Enter 'hit' or 'stand'): stand
Your hand: ['6 of Spades', '3 of Spades', 'King of Spades'] (19)
Dealer's hand: ['Jack of Spades', 'King of Hearts'] (20)
Dealer wins!


5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

In [8]:
import random

# Define the cards and deck
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_decks):
        self.cards = [Card(suit, rank) for _ in range(num_decks) for suit in suits for rank in ranks]
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def deal_card(self):
        if len(self.cards) == 0:
            raise ValueError("No cards left in the deck.")
        return self.cards.pop()

class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def get_value(self):
        value = 0
        ace_count = 0

        for card in self.cards:
            if card.rank.isdigit():
                value += int(card.rank)
            elif card.rank in ['Jack', 'Queen', 'King']:
                value += 10
            else:
                ace_count += 1
                value += 11

        while value > 21 and ace_count:
            value -= 10
            ace_count -= 1

        return value

# Player classes
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = Hand()

    def receive_card(self, card):
        self.hand.add_card(card)

class HumanPlayer(Player):
    def make_move(self):
        return input(f"{self.name}, do you want to hit or stand? (Enter 'hit' or 'stand'): ").lower()

class Dealer(Player):
    def play_turn(self, deck):
        while self.hand.get_value() < 17:
            self.receive_card(deck.deal_card())

# Game Logic
def blackjack():
    num_decks = 6
    deck = Deck(num_decks)

    num_players = 3
    players = [HumanPlayer(f"Player {i+1}") for i in range(num_players)]
    dealer = Dealer("Dealer")

    for round_num in range(1, 4):  # Play 3 rounds
        print(f"----- Round {round_num} -----")
        for player in players + [dealer]:
            for _ in range(2):  # Deal initial 2 cards to each player
                player.receive_card(deck.deal_card())

            if isinstance(player, HumanPlayer):
                while True:
                    move = player.make_move()
                    if move == 'hit':
                        player.receive_card(deck.deal_card())
                        if player.hand.get_value() > 21:
                            print(f"{player.name} busted! Dealer wins.")
                            break
                    else:
                        break
            else:
                player.play_turn(deck)

            print(f"{player.name}'s hand: {[str(card) for card in player.hand.cards]} ({player.hand.get_value()})")

    # Determine winner
    dealer_value = dealer.hand.get_value()
    for player in players:
        player_value = player.hand.get_value()

        print(f"{player.name}'s hand: {[str(card) for card in player.hand.cards]} ({player_value})")
        if player_value > 21:
            print(f"{player.name} busted! Dealer wins.")
        elif dealer_value > 21 or player_value > dealer_value:
            print(f"{player.name} wins!")
        elif dealer_value > player_value:
            print(f"{player.name} loses.")
        else:
            print(f"{player.name} and dealer tied.")

# Start the game
blackjack()


----- Round 1 -----
Player 1, do you want to hit or stand? (Enter 'hit' or 'stand'): hit
Player 1, do you want to hit or stand? (Enter 'hit' or 'stand'): stand
Player 1's hand: ['Ace of Diamonds', '8 of Spades', '9 of Clubs'] (18)
Player 2, do you want to hit or stand? (Enter 'hit' or 'stand'): hit
Player 2 busted! Dealer wins.
Player 2's hand: ['10 of Clubs', '10 of Diamonds', '5 of Spades'] (25)
Player 3, do you want to hit or stand? (Enter 'hit' or 'stand'): stand
Player 3's hand: ['10 of Hearts', '8 of Hearts'] (18)
Dealer's hand: ['10 of Diamonds', 'Ace of Clubs'] (21)
----- Round 2 -----
Player 1, do you want to hit or stand? (Enter 'hit' or 'stand'): hit
Player 1 busted! Dealer wins.
Player 1's hand: ['Ace of Diamonds', '8 of Spades', '9 of Clubs', '8 of Diamonds', '9 of Spades', '10 of Hearts'] (45)
Player 2, do you want to hit or stand? (Enter 'hit' or 'stand'): stand
Player 2's hand: ['10 of Clubs', '10 of Diamonds', '5 of Spades', '6 of Spades', 'Jack of Spades'] (41)
Player

KeyboardInterrupt: Interrupted by user

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

In [ ]:
import random

# Define the cards and deck
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_decks):
        self.cards = [Card(suit, rank) for _ in range(num_decks) for suit in suits for rank in ranks]
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def deal_card(self):
        if len(self.cards) == 0:
            raise ValueError("No cards left in the deck.")
        return self.cards.pop()

class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def get_value(self):
        value = 0
        ace_count = 0

        for card in self.cards:
            if card.rank.isdigit():
                value += int(card.rank)
            elif card.rank in ['Jack', 'Queen', 'King']:
                value += 10
            else:
                ace_count += 1
                value += 11

        while value > 21 and ace_count:
            value -= 10
            ace_count -= 1

        return value

# Player classes
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = Hand()

    def receive_card(self, card):
        self.hand.add_card(card)

class HumanPlayer(Player):
    def make_move(self):
        return input(f"{self.name}, do you want to hit or stand? (Enter 'hit' or 'stand'): ").lower()

class Dealer(Player):
    def play_turn(self, deck):
        while self.hand.get_value() < 17:
            self.receive_card(deck.deal_card())

# Game Logic
def blackjack():
    num_decks = 6
    deck = Deck(num_decks)

    num_players = 3
    players = [HumanPlayer(f"Player {i+1}") for i in range(num_players)]
    dealer = Dealer("Dealer")

    for round_num in range(1, 4):  # Play 3 rounds
        print(f"----- Round {round_num} -----")
        for player in players + [dealer]:
            for _ in range(2):  # Deal initial 2 cards to each player
                player.receive_card(deck.deal_card())

            if isinstance(player, HumanPlayer):
                while True:
                    move = player.make_move()
                    if move == 'hit':
                        player.receive_card(deck.deal_card())
                        if player.hand.get_value() > 21:
                            print(f"{player.name} busted! Dealer wins.")
                            break
                    else:
                        break
            else:
                player.play_turn(deck)

            print(f"{player.name}'s hand: {[str(card) for card in player.hand.cards]} ({player.hand.get_value()})")

    # Determine winner
    dealer_value = dealer.hand.get_value()
    for player in players:
        player_value = player.hand.get_value()

        print(f"{player.name}'s hand: {[str(card) for card in player.hand.cards]} ({player_value})")
        if player_value > 21:
            print(f"{player.name} busted! Dealer wins.")
        elif dealer_value > 21 or player_value > dealer_value:
            print(f"{player.name} wins!")
        elif dealer_value > player_value:
            print(f"{player.name} loses.")
        else:
            print(f"{player.name} and dealer tied.")

# Start the game
blackjack()


----- Round 1 -----
Player 1, do you want to hit or stand? (Enter 'hit' or 'stand'): hit
Player 1 busted! Dealer wins.
Player 1's hand: ['9 of Hearts', '5 of Diamonds', '9 of Spades'] (23)
Player 2, do you want to hit or stand? (Enter 'hit' or 'stand'): stand
Player 2's hand: ['8 of Diamonds', '10 of Diamonds'] (18)
Player 3, do you want to hit or stand? (Enter 'hit' or 'stand'): hit
Player 3, do you want to hit or stand? (Enter 'hit' or 'stand'): stand
Player 3's hand: ['2 of Clubs', '8 of Hearts', 'Ace of Clubs'] (21)
Dealer's hand: ['5 of Hearts', '10 of Spades', '7 of Hearts'] (22)
----- Round 2 -----
Player 1, do you want to hit or stand? (Enter 'hit' or 'stand'): jit
Player 1's hand: ['9 of Hearts', '5 of Diamonds', '9 of Spades', '4 of Hearts', '6 of Diamonds'] (33)
Player 2, do you want to hit or stand? (Enter 'hit' or 'stand'): stand
Player 2's hand: ['8 of Diamonds', '10 of Diamonds', '8 of Diamonds', 'Queen of Spades'] (36)


7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

In [ ]:
import random

# Define the cards and deck
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        return f"{self.rank} of {self.suit}"

class Deck:
    def __init__(self, num_decks):
        self.cards = [Card(suit, rank) for _ in range(num_decks) for suit in suits for rank in ranks]
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def deal_card(self):
        if len(self.cards) == 0:
            raise ValueError("No cards left in the deck.")
        return self.cards.pop()

class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def get_value(self):
        value = 0
        ace_count = 0

        for card in self.cards:
            if card.rank.isdigit():
                value += int(card.rank)
            elif card.rank in ['Jack', 'Queen', 'King']:
                value += 10
            else:
                ace_count += 1
                value += 11

        while value > 21 and ace_count:
            value -= 10
            ace_count -= 1

        return value

class Player:
    def __init__(self, name, chips):
        self.name = name
        self.chips = chips
        self.hand = Hand()

    def receive_card(self, card):
        self.hand.add_card(card)

class CardCountingPlayer(Player):
    def __init__(self, name, chips, threshold):
        super().__init__(name, chips)
        self.threshold = threshold
        self.count = 0

    def count_cards(self, card):
        if card.rank.isdigit():
            if 2 <= int(card.rank) <= 6:
                self.count += 1
            elif 7 <= int(card.rank) <= 9:
                pass  # Cards 7 to 9 are neutral (value 0)
            else:
                self.count -= 1
        else:
            self.count -= 1

    def make_move(self):
        if self.count <= self.threshold:
            return 'hit'
        else:
            return 'stand'

# Game Logic
def play_blackjack():
    num_decks = 6
    deck = Deck(num_decks)

    strategy_player = CardCountingPlayer("Strategy Player", 1000, -2)
    dealer = Player("Dealer", 1000)
    other_players = [Player(f"Player {i+1}", 1000) for i in range(3)]

    rounds = 0
    while rounds < 50 and strategy_player.chips > 0:
        rounds += 1

        # Deal initial cards
        for player in [strategy_player] + other_players + [dealer]:
            for _ in range(2):
                player.receive_card(deck.deal_card())
                if isinstance(player, CardCountingPlayer):
                    player.count_cards(player.hand.cards[-1])

        # Strategy player's turn
        move = strategy_player.make_move()
        while move == 'hit':
            strategy_player.receive_card(deck.deal_card())
            strategy_player.count_cards(strategy_player.hand.cards[-1])
            if strategy_player.hand.get_value() > 21:
                strategy_player.chips -= 10
                break
            else:
                move = strategy_player.make_move()

        # Dealer's turn
        while dealer.hand.get_value() < 17:
            dealer.receive_card(deck.deal_card())

        # Determine winners and adjust chips
        for player in other_players:
            player_value = player.hand.get_value()
            dealer_value = dealer.hand.get_value()

            if player_value > 21 or (dealer_value <= 21 and dealer_value > player_value):
                player.chips -= 10
            elif dealer_value > 21 or player_value > dealer_value:
                player.chips += 10

        # Strategy player's chips after the round
        strategy_player_value = strategy_player.hand.get_value()
        dealer_value = dealer.hand.get_value()

        if strategy_player_value > 21 or (dealer_value <= 21 and dealer_value > strategy_player_value):
            strategy_player.chips -= 10
        elif dealer_value > 21 or strategy_player_value > dealer_value:
            strategy_player.chips += 10

    return strategy_player.chips

# Simulate the game
final_chips = play_blackjack()
print(f"The strategy player's final chips after 50 rounds: {final_chips}")


8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt



# Function to play a single game of 50 rounds
def play_50_rounds(deck, strategy_player, dealer, other_players):
    rounds = 0
    while rounds < 50 and strategy_player.chips > 0:
        rounds += 1

        # Deal initial cards
        for player in [strategy_player] + other_players + [dealer]:
            for _ in range(2):
                player.receive_card(deck.deal_card())
                if isinstance(player, CardCountingPlayer):
                    player.count_cards(player.hand.cards[-1])

        # Strategy player's turn
        move = strategy_player.make_move()
        while move == 'hit':
            strategy_player.receive_card(deck.deal_card())
            strategy_player.count_cards(strategy_player.hand.cards[-1])
            if strategy_player.hand.get_value() > 21:
                strategy_player.chips -= 10
                break
            else:
                move = strategy_player.make_move()

        # Dealer's turn
        while dealer.hand.get_value() < 17:
            dealer.receive_card(deck.deal_card())

        # Determine winners and adjust chips
        for player in other_players:
            player_value = player.hand.get_value()
            dealer_value = dealer.hand.get_value()

            if player_value > 21 or (dealer_value <= 21 and dealer_value > player_value):
                player.chips -= 10
            elif dealer_value > 21 or player_value > dealer_value:
                player.chips += 10

        # Strategy player's chips after the round
        strategy_player_value = strategy_player.hand.get_value()
        dealer_value = dealer.hand.get_value()

        if strategy_player_value > 21 or (dealer_value <= 21 and dealer_value > strategy_player_value):
            strategy_player.chips -= 10
        elif dealer_value > 21 or strategy_player_value > dealer_value:
            strategy_player.chips += 10

    return strategy_player.chips

# Function to play 100 games of 50 rounds each
def simulate_games():
    num_games = 100
    strategy_player_winnings = []

    for _ in range(num_games):
        # Initialize players and deck for each game
        deck = Deck(6)
        strategy_player = CardCountingPlayer("Strategy Player", 1000, -2)
        dealer = Player("Dealer", 1000)
        other_players = [Player(f"Player {i+1}", 1000) for i in range(3)]

        # Play a game and store strategy player's chips
        final_chips = play_50_rounds(deck, strategy_player, dealer, other_players)
        strategy_player_winnings.append(final_chips)

    return strategy_player_winnings

# Run 100 games and store strategy player's winnings
strategy_player_winnings = simulate_games()

# Histogram of strategy player's winnings
plt.hist(strategy_player_winnings, bins=20, edgecolor='black')
plt.xlabel('Strategy Player Winnings')
plt.ylabel('Frequency')
plt.title('Histogram of Strategy Player Winnings after 100 games of 50 rounds each')
plt.show()

# Calculate statistics
average_winnings_per_round = np.mean(strategy_player_winnings) / 50
standard_deviation = np.std(strategy_player_winnings)
net_winning_probability = sum(winnings > 1000 for winnings in strategy_player_winnings) / len(strategy_player_winnings)

print(f"Average winnings per round: {average_winnings_per_round}")
print(f"Standard deviation of winnings: {standard_deviation}")
print(f"Probability of net winning after 50 rounds: {net_winning_probability}")


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

In [ ]:
def simulate_games_for_threshold(threshold):
    num_games = 100
    strategy_player_winnings = []

    for _ in range(num_games):
        deck = Deck(6)
        strategy_player = CardCountingPlayer("Strategy Player", 1000, threshold)
        dealer = Player("Dealer", 1000)
        other_players = [Player(f"Player {i+1}", 1000) for i in range(3)]

        final_chips = play_50_rounds(deck, strategy_player, dealer, other_players)
        strategy_player_winnings.append(final_chips)

    return strategy_player_winnings

threshold_values = [-2, -1, 0, 1, 2]  # Try different threshold values
winning_stats_per_threshold = {}

for threshold in threshold_values:
    strategy_player_winnings = simulate_games_for_threshold(threshold)
    winning_stats_per_threshold[threshold] = strategy_player_winnings

# Display statistics for different threshold values
for threshold, winnings in winning_stats_per_threshold.items():
    average_winnings_per_round = np.mean(winnings) / 50
    standard_deviation = np.std(winnings)
    net_winning_probability = sum(winnings > 1000 for winnings in winnings) / len(winnings)

    print(f"Threshold: {threshold}")
    print(f"Average winnings per round: {average_winnings_per_round}")
    print(f"Standard deviation of winnings: {standard_deviation}")
    print(f"Probability of net winning after 50 rounds: {net_winning_probability}")
    print("-------------------------------")


10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 

In [ ]:


class BasicStrategyPlayer(Player):
    def make_move(self, dealer_upcard):
        player_hand_value = self.hand.get_value()

        if player_hand_value >= 17:
            return 'stand'
        elif player_hand_value <= 11:
            return 'hit'
        elif player_hand_value == 12:
            return 'stand' if 4 <= dealer_upcard.get_value() <= 6 else 'hit'
        elif 13 <= player_hand_value <= 16:
            return 'stand' if 2 <= dealer_upcard.get_value() <= 6 else 'hit'
        else:
            return 'hit'

# Function to play a single game of 50 rounds with Basic Strategy player
def play_50_rounds_with_basic_strategy(deck, basic_strategy_player, dealer, other_players):
    rounds = 0
    while rounds < 50 and basic_strategy_player.chips > 0:
        rounds += 1

        # Deal initial cards
        for player in [basic_strategy_player] + other_players + [dealer]:
            for _ in range(2):
                player.receive_card(deck.deal_card())

        # Basic Strategy player's turn
        move = basic_strategy_player.make_move(dealer.hand.cards[0])
        while move == 'hit':
            basic_strategy_player.receive_card(deck.deal_card())
            move = basic_strategy_player.make_move(dealer.hand.cards[0])
            if basic_strategy_player.hand.get_value() > 21:
                basic_strategy_player.chips -= 10
                break

        # Dealer's turn
        while dealer.hand.get_value() < 17:
            dealer.receive_card(deck.deal_card())

        # Determine winners and adjust chips
        for player in other_players:
            player_value = player.hand.get_value()
            dealer_value = dealer.hand.get_value()

            if player_value > 21 or (dealer_value <= 21 and dealer_value > player_value):
                player.chips -= 10
            elif dealer_value > 21 or player_value > dealer_value:
                player.chips += 10

        # Basic Strategy player's chips after the round
        player_value = basic_strategy_player.hand.get_value()
        dealer_value = dealer.hand.get_value()

        if player_value > 21 or (dealer_value <= 21 and dealer_value > player_value):
            basic_strategy_player.chips -= 10
        elif dealer_value > 21 or player_value > dealer_value:
            basic_strategy_player.chips += 10

    return basic_strategy_player.chips

# Run simulations for Basic Strategy player
def simulate_games_with_basic_strategy():
    num_games = 100
    basic_strategy_player_winnings = []

    for _ in range(num_games):
        deck = Deck(6)
        basic_strategy_player = BasicStrategyPlayer("Basic Strategy Player", 1000)
        dealer = Player("Dealer", 1000)
        other_players = [Player(f"Player {i+1}", 1000) for i in range(3)]

        final_chips = play_50_rounds_with_basic_strategy(deck, basic_strategy_player, dealer, other_players)
        basic_strategy_player_winnings.append(final_chips)

    return basic_strategy_player_winnings

# Run simulations for Basic Strategy player
basic_strategy_player_winnings = simulate_games_with_basic_strategy()

# Analyze statistics for Basic Strategy player
average_winnings_per_round = np.mean(basic_strategy_player_winnings) / 50
standard_deviation = np.std(basic_strategy_player_winnings)
net_winning_probability = sum(winnings > 1000 for winnings in basic_strategy_player_winnings) / len(basic_strategy_player_winnings)

print(f"Basic Strategy Player:")
print(f"Average winnings per round: {average_winnings_per_round}")
print(f"Standard deviation of winnings: {standard_deviation}")
print(f"Probability of net winning after 50 rounds: {net_winning_probability}")
